# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [45]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [46]:
!pip install gmaps

In [47]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [49]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Srandakan,ID,-7.9386,110.2506,82.26,75,100,1.81,overcast clouds
1,1,Saldanha,ZA,-33.0117,17.9442,59.05,87,7,8.01,clear sky
2,2,Saskylakh,RU,71.9167,114.0833,-18.81,95,92,6.04,overcast clouds
3,3,Victoria,HK,22.2855,114.1577,78.85,79,100,8.99,overcast clouds
4,4,Balagon,PH,8.0063,123.2396,81.03,69,100,2.98,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [50]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimun temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 65
What is the maximun temperature you would like for your trip? 85


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [51]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Srandakan,ID,-7.9386,110.2506,82.26,75,100,1.81,overcast clouds
3,3,Victoria,HK,22.2855,114.1577,78.85,79,100,8.99,overcast clouds
4,4,Balagon,PH,8.0063,123.2396,81.03,69,100,2.98,overcast clouds
5,5,San Luis,AR,-33.2950,-66.3356,65.89,36,0,9.33,clear sky
8,8,Mataura,NZ,-46.1927,168.8643,73.17,39,98,3.98,overcast clouds
9,9,Vila Franca Do Campo,PT,37.7167,-25.4333,65.19,92,56,4.34,broken clouds
12,12,Saint-Philippe,RE,-21.3585,55.7679,69.39,68,39,12.68,scattered clouds
18,18,Kapaa,US,22.0752,-159.3190,82.38,72,40,14.97,scattered clouds
19,19,Oranjestad,AW,12.5240,-70.0270,82.35,87,20,11.50,few clouds
20,20,Atuona,PF,-9.8000,-139.0333,77.74,76,4,14.90,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [52]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data

clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Srandakan,ID,-7.9386,110.2506,82.26,75,100,1.81,overcast clouds
3,3,Victoria,HK,22.2855,114.1577,78.85,79,100,8.99,overcast clouds
4,4,Balagon,PH,8.0063,123.2396,81.03,69,100,2.98,overcast clouds
5,5,San Luis,AR,-33.2950,-66.3356,65.89,36,0,9.33,clear sky
8,8,Mataura,NZ,-46.1927,168.8643,73.17,39,98,3.98,overcast clouds


In [53]:
clean_travel_cities.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [54]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
0,Srandakan,ID,82.26,overcast clouds,-7.9386,110.2506
3,Victoria,HK,78.85,overcast clouds,22.2855,114.1577
4,Balagon,PH,81.03,overcast clouds,8.0063,123.2396
5,San Luis,AR,65.89,clear sky,-33.2950,-66.3356
8,Mataura,NZ,73.17,overcast clouds,-46.1927,168.8643
...,...,...,...,...,...,...
692,Acajutla,SV,79.16,broken clouds,13.5928,-89.8275
697,Axim,GH,76.93,light rain,4.8699,-2.2405
699,Birao,CF,67.26,broken clouds,10.2849,22.7882
701,Iranshahr,IR,67.69,clear sky,27.2025,60.6848


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [55]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head(10)

C:\Users\checo\AppData\Local\Temp/ipykernel_16604/19374355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Srandakan,ID,82.26,overcast clouds,-7.9386,110.2506,
3,Victoria,HK,78.85,overcast clouds,22.2855,114.1577,
4,Balagon,PH,81.03,overcast clouds,8.0063,123.2396,
5,San Luis,AR,65.89,clear sky,-33.2950,-66.3356,
8,Mataura,NZ,73.17,overcast clouds,-46.1927,168.8643,
9,Vila Franca Do Campo,PT,65.19,broken clouds,37.7167,-25.4333,
12,Saint-Philippe,RE,69.39,scattered clouds,-21.3585,55.7679,
18,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
19,Oranjestad,AW,82.35,few clouds,12.5240,-70.0270,
20,Atuona,PF,77.74,clear sky,-9.8000,-139.0333,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [56]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        



## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [57]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng =row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")

C:\Users\checo\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [58]:
# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Srandakan,ID,82.26,overcast clouds,-7.9386,110.2506,Puri Brata
3,Victoria,HK,78.85,overcast clouds,22.2855,114.1577,Mini Central
4,Balagon,PH,81.03,overcast clouds,8.0063,123.2396,Casa maria molave
5,San Luis,AR,65.89,clear sky,-33.2950,-66.3356,Huarpes Hotel
8,Mataura,NZ,73.17,overcast clouds,-46.1927,168.8643,Ellie's Villa
...,...,...,...,...,...,...,...
692,Acajutla,SV,79.16,broken clouds,13.5928,-89.8275,CASA DE LA PERRA
697,Axim,GH,76.93,light rain,4.8699,-2.2405,Axim Beach Resort And Spa
699,Birao,CF,67.26,broken clouds,10.2849,22.7882,
701,Iranshahr,IR,67.69,clear sky,27.2025,60.6848,نیک سرشت


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [59]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Srandakan,ID,82.26,overcast clouds,-7.9386,110.2506,Puri Brata
3,Victoria,HK,78.85,overcast clouds,22.2855,114.1577,Mini Central
4,Balagon,PH,81.03,overcast clouds,8.0063,123.2396,Casa maria molave
5,San Luis,AR,65.89,clear sky,-33.2950,-66.3356,Huarpes Hotel
8,Mataura,NZ,73.17,overcast clouds,-46.1927,168.8643,Ellie's Villa
...,...,...,...,...,...,...,...
692,Acajutla,SV,79.16,broken clouds,13.5928,-89.8275,CASA DE LA PERRA
697,Axim,GH,76.93,light rain,4.8699,-2.2405,Axim Beach Resort And Spa
699,Birao,CF,67.26,broken clouds,10.2849,22.7882,
701,Iranshahr,IR,67.69,clear sky,27.2025,60.6848,نیک سرشت


In [60]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [61]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [62]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
print(info_box_template)


<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>



## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [63]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Puri Brata</dd>\n<dt>City</dt><dd>Srandakan</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 82.26 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Mini Central</dd>\n<dt>City</dt><dd>Victoria</dd>\n<dt>Country</dt><dd>HK</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 78.85 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Casa maria molave</dd>\n<dt>City</dt><dd>Balagon</dd>\n<dt>Country</dt><dd>PH</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 81.03 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Huarpes Hotel</dd>\n<dt>City</dt><dd>San Luis</dd>\n<dt>Country</dt><dd>AR</dd>\n<dt>Current Weather</dt><dd>clear sky and 65.89 °F</dd>\n</dl>\n',
 "\n<dl>\n<dt>Hotel Name</dt><dd>Ellie's Villa</dd>\n<dt>City</dt><dd>Mataura</dd>\n<dt>Country</dt><dd>NZ</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 73.17 °F</dd>\n</dl>\n",
 '\n<dl>\n<dt>Hotel Name</dt><dd>Pestana Bahia Praia</dd>\n<dt>City</dt>

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [64]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-7.9386,110.2506
3,22.2855,114.1577
4,8.0063,123.2396
5,-33.2950,-66.3356
8,-46.1927,168.8643
9,37.7167,-25.4333
12,-21.3585,55.7679
18,22.0752,-159.3190
19,12.5240,-70.0270
20,-9.8000,-139.0333


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [20]:
# 11a. Add a marker layer for each city to the map. 
description=hotel_df["Current Description"]
fig= gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, dissipating= False,
                               max_intensity=300, point_radius=4)
marker_layer= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

# fig.savefig("../Vacation_Search/Resources/WeatherPy_vacation_map.png")

Figure(layout=FigureLayout(height='420px'))

In [66]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Create a figure to add the Google map as a layer
fig.add_layer(marker_layer)

# Display the figure containing the map
fig
# Save map
# plt.savefig("../Vacation_Search/Resources/WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>